Licensed under the Apache License, Version 2.0

In [ ]:
#@title Install dependencies
!pip install xarray
!pip install gcsfs
!pip install pyresample

In [ ]:
import datetime
import pprint

import gcsfs
import matplotlib.pyplot as plt
import numpy as np
import pyresample
import xarray

# Load GOES data

GOES filenames contain lots of information, with a few important parts.

`OR_ABI-L1b-RadF-M6C11_G16_s20230010020206_e20230010029514_c20230010029565.nc`
* `L1b` - Level-1b data product, as opposed to L2 which are derived data.
* `RadF` - Full-disk radiances.
* `C11` - Data for band #11.
* `s20230010020` - Start time of the GOES scan. Scans happen every 10 minutes, so this scan happened on:
  * `2023` - year
  * `001` - day of year, e.g. Jan 1st
  * `00` - hour, `20` - minute

Here we load 3 bands from this scan - `11`, `14`, and `15` and convert the radiances to brightness temperatures. These 3 bands are required for our color scheme.

GOES data is hosted on Google Cloud for public use.

References:
* GOES data products: https://www.goes-r.gov/products/overview.html
* GOES public dataset: https://console.cloud.google.com/marketplace/product/noaa-public/goes-16

In [ ]:
fs = gcsfs.GCSFileSystem(project='gcp-public-data-goes-16')
print('All band paths for scan:')
pprint.pprint(fs.glob('gcp-public-data-goes-16/ABI-L1b-RadF/2023/001/00/*_s20230010020*'))

paths = {
    11: 'gcp-public-data-goes-16/ABI-L1b-RadF/2023/001/00/OR_ABI-L1b-RadF-M6C11_G16_s20230010020206_e20230010029514_c20230010029565.nc',
    14: 'gcp-public-data-goes-16/ABI-L1b-RadF/2023/001/00/OR_ABI-L1b-RadF-M6C14_G16_s20230010020206_e20230010029514_c20230010029555.nc',
    15: 'gcp-public-data-goes-16/ABI-L1b-RadF/2023/001/00/OR_ABI-L1b-RadF-M6C15_G16_s20230010020206_e20230010029522_c20230010029573.nc'
}

brightness_temperatures = {}
for band_id, path in paths.items():
  with fs.open(path, 'rb') as f:
    dataset = xarray.open_dataset(f)
    # Convert radiances to brightness temperature
    brightness_temperatures[band_id] = (dataset.planck_fk2.data / np.log((dataset.planck_fk1.data / dataset.Rad.data) + 1) - dataset.planck_bc1.data) / dataset.planck_bc2.data


# Generate false color image

In order to view contrails in GOES, we use the "ash" color scheme. This color scheme was originally developed for viewing volcanic ash in the atmosphere but is also useful for viewing thin cirrus, including contrails. In this color scheme, contrails appear in the image as dark blue.

Note that we use a modified version of the ash color scheme here, developed by Kulik et al., which uses slightly different bands and bounds tuned for contrails.

References:
* Original Ash RGB description: https://rammb.cira.colostate.edu/training/visit/quick_guides/GOES_Ash_RGB.pdf
* Modified Ash Color Scheme (Kulik et al., page 22): https://dspace.mit.edu/handle/1721.1/124179?show=full

In [ ]:
_T11_BOUNDS = (243, 303)
_CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
_TDIFF_BOUNDS = (-4, 2)

def normalize_range(data, bounds):
  """Maps data to the range [0, 1]."""
  return (data - bounds[0]) / (bounds[1] - bounds[0])

def false_color_image(brightness_temperatures):
  """Generates ash false color image from GOES brightness temperatures."""
  r = normalize_range(brightness_temperatures[15] - brightness_temperatures[14], _TDIFF_BOUNDS)
  g = normalize_range(brightness_temperatures[14] - brightness_temperatures[11], _CLOUD_TOP_TDIFF_BOUNDS)
  b = normalize_range(brightness_temperatures[14], _T11_BOUNDS)
  return np.clip(np.stack([r, g, b], axis=-1), 0, 1)

In [ ]:
fci = false_color_image(brightness_temperatures)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(fci)

# Find a location on the image

The GOES data comes with information about the projection, which we can use to find a lat/lng location on the image. Note that we are *not* considering parallax here, so the lat/lng location that we plot will be on the surface.

In this example, we find New York City on the image.

In [ ]:
NYC_LAT = 40.7128
NYC_LNG = -74.0060

In [ ]:
# Open the netCDF file for one of the bands. All bands will have the same projection information.
with fs.open(paths[11], 'rb') as f:
  dataset = xarray.open_dataset(f)

  h0 = dataset.goes_imager_projection.perspective_point_height
  area_def = pyresample.geometry.AreaDefinition(
    area_id='all_goes_16',  # Used only for pyresample logging
    proj_id='deprecated',  # Deprecated but required by pyresample
    description='all_goes_16',  # Used only for pyresample logging
    projection={  # proj4 dict
        'proj': 'geos',  # Stands for 'geostationary'
        'units': 'm',
        'h': str(h0),
        'lon_0': str(
            dataset.goes_imager_projection.longitude_of_projection_origin
        ),
        'a': str(dataset.goes_imager_projection.semi_major_axis),
        'b': str(dataset.goes_imager_projection.semi_minor_axis),
        'sweep': dataset.goes_imager_projection.sweep_angle_axis,
    },
    width=dataset['x'].shape[0],
    height=dataset['y'].shape[0],
    area_extent=[
        dataset['x_image_bounds'].data[0] * h0,
        dataset['y_image_bounds'].data[1] * h0,
        dataset['x_image_bounds'].data[1] * h0,
        dataset['y_image_bounds'].data[0] * h0,
    ],
  )
# Convert the lat/lng to row/col indices in the image.
col, row = area_def.lonlat2colrow(NYC_LNG, NYC_LAT)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(col, row, marker='x', color='red', markersize=12, markeredgewidth=5)
plt.imshow(fci)
plt.show()